In [ ]:
# ================================
# 1. 필수 라이브러리 임포트 섹션
# ================================

import torch  # PyTorch 메인 라이브러리 - 텐서 연산 및 자동 미분
import torch.nn as nn  # PyTorch 신경망 모듈 - 레이어, 활성화 함수, 손실 함수 등

import numpy as np  # 수치 연산 라이브러리 - 배열 연산, 수학 함수
import glob  # 파일 경로 패턴 매칭을 위한 glob 모듈 임포트 (예: *.png 파일 찾기)
import os  # 운영체제 기능(파일 시스템 등)을 사용하기 위한 os 모듈 임포트

import torchvision  # PyTorch 컴퓨터 비전 라이브러리 - 이미지 처리 및 데이터셋
from torchvision import datasets  # 데이터셋 로딩을 위한 datasets 모듈 임포트 (MNIST, CIFAR 등)
import torchvision.transforms as transforms  # 이미지 변환을 위한 transforms 모듈 임포트 (크기 조정, 정규화 등)
from torchvision.utils import save_image  # 이미지 저장을 위한 save_image 함수 임포트
from torch.autograd import Variable  # 자동 미분을 위한 Variable 임포트 (현재는 권장되지 않음)
import matplotlib.pyplot as plt  # 이미지 시각화를 위한 matplotlib.pyplot 임포트
from IPython.display import Image  # 주피터 노트북에 이미지 표시를 위한 Image 클래스 임포트
from IPython.display import display  # 주피터 노트북에 객체 표시를 위한 display 함수 임포트

# ================================
# 2. Google Drive 마운트 (Colab 환경)
# ================================

from google.colab import drive  # Google Colab에서 Drive 연결을 위한 모듈
drive.mount('/content/drive')  # Google Drive를 /content/drive 경로에 마운트

# ================================
# 3. 데이터 전처리 및 로딩 설정
# ================================

batch_size = 64  # 학습에 사용할 배치 크기 설정 - 한 번에 처리할 이미지 수

# 이미지 전처리 파이프라인 정의
transforms_train = transforms.Compose([
    transforms.Resize(28),  # 이미지 크기를 28x28로 조정 (MNIST는 원래 28x28이지만 명시적 설정)
    transforms.ToTensor(),  # PIL Image나 numpy 배열을 PyTorch 텐서로 변환 (픽셀 값 [0, 1]로 정규화)
    transforms.Normalize([0.5], [0.5])  # 이미지를 평균 0.5, 표준편차 0.5로 정규화 (픽셀 값 [-1, 1] 범위로 변환)
])
# 정규화 공식: (pixel - 0.5) / 0.5 = 2*pixel - 1
# [0, 1] 범위가 [-1, 1] 범위로 변환됨 (Generator의 Tanh 출력과 매칭)

# MNIST 데이터셋 로드 및 전처리 적용
train_dataset = datasets.MNIST(
    root="./dataset",  # 데이터를 저장할 로컬 디렉토리
    train=True,  # 학습용 데이터셋 사용 (60,000개 이미지)
    download=True,  # 데이터가 없으면 자동 다운로드
    transform=transforms_train  # 위에서 정의한 전처리 파이프라인 적용
)

# 데이터 로더 생성 - 배치 단위로 데이터를 효율적으로 로딩
dataloader = torch.utils.data.DataLoader(
    train_dataset,  # 사용할 데이터셋
    batch_size=batch_size,  # 배치 크기 (64)
    shuffle=True,  # 에포크마다 데이터 순서를 무작위로 섞음 (학습 성능 향상)
    num_workers=4  # 데이터 로딩을 위한 멀티 프로세스 워커 수 (병렬 처리로 속도 향상)
)

# ================================
# 4. 데이터 시각화 (학습 전 확인)
# ================================

# 데이터 로더에서 첫 번째 이미지 배치와 레이블 가져오기
images, labels = next(iter(dataloader))

# 이미지 배치를 하나의 그리드 이미지로 합치기
img = torchvision.utils.make_grid(images)  # 여러 이미지를 격자 형태로 배열

# PyTorch 텐서를 NumPy 배열로 변환하고 차원 순서 변경
# PyTorch: (C, H, W) -> Matplotlib: (H, W, C)
img = img.numpy().transpose(1, 2, 0)

# 이미지 정규화에 사용된 표준편차와 평균 (시각화를 위한 역정규화에 사용)
std = [0.5, 0.5, 0.5]  # RGB 각 채널의 표준편차 (MNIST는 흑백이지만 3채널로 표시)
mean = [0.5, 0.5, 0.5]  # RGB 각 채널의 평균

# 이미지 역정규화 수행 - 정규화된 [-1, 1] 범위를 [0, 1] 범위로 되돌림
img = img * std + mean  # 역정규화 공식: denormalized = normalized * std + mean

# 첫 번째 배치에 포함된 이미지들의 레이블 출력 (0~9 숫자)
print([labels[i] for i in range(batch_size)])

# 이미지 그리드 시각화 - matplotlib으로 이미지 표시
plt.imshow(img)

# ================================
# 5. 모델 하이퍼파라미터 설정
# ================================

# 이미지 관련 설정
channels = 1  # 이미지 채널 수 (MNIST는 흑백이므로 1채널)
img_size = 28  # 이미지 크기 (높이 및 너비 모두 28픽셀)
img_shape = (channels, img_size, img_size)  # 이미지 형태를 튜플로 정의 (1, 28, 28)

# ================================
# 6. Generator(생성자) 모델 정의
# ================================

class Generator(nn.Module):  # 생성자 모델 클래스 정의 (nn.Module 상속)
    def __init__(self):  # 생성자 초기화 함수
        super(Generator, self).__init__()  # 부모 클래스(nn.Module) 초기화

        # 생성자 내부에서 사용할 선형 블록을 정의하는 중첩 함수
        def block(input_dim, output_dim, normalize=True):
            """
            선형 변환 블록을 생성하는 함수
            Args:
                input_dim: 입력 차원
                output_dim: 출력 차원
                normalize: 배치 정규화 적용 여부
            Returns:
                레이어들의 리스트
            """
            layers = [nn.Linear(input_dim, output_dim)]  # 입력 차원에서 출력 차원으로 변환하는 선형 레이어

            if normalize:
                # 배치 정규화 레이어 추가 (학습 안정화 및 수렴 속도 향상)
                # 0.8은 momentum 파라미터 (이동평균 계산 시 가중치)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))

            # LeakyReLU 활성화 함수 추가
            # 0.2는 negative slope (음수 구간에서의 기울기)
            # inplace=True는 메모리 효율성을 위해 입력을 직접 수정
            layers.append(nn.LeakyReLU(0.2, inplace=True))

            return layers

        # 여러 레이어를 순차적으로 연결하는 Sequential 모델 구성
        self.model = nn.Sequential(
            # 첫 번째 블록: 잠재 공간(latent_dim)에서 128차원으로 확장
            # normalize=False: 첫 번째 레이어에서는 배치 정규화 생략
            *block(latent_dim, output_dim=128, normalize=False),

            # 두 번째 블록: 128차원에서 256차원으로 확장
            *block(128, 256),

            # 세 번째 블록: 256차원에서 512차원으로 확장
            *block(256, 512),

            # 네 번째 블록: 512차원에서 1024차원으로 확장
            *block(512, 1024),

            # 최종 출력 레이어: 1024차원에서 이미지 픽셀 수(784)로 변환
            # np.prod(img_shape) = 1 * 28 * 28 = 784
            nn.Linear(1024, int(np.prod(img_shape))),

            # Tanh 활성화 함수: 출력 값을 [-1, 1] 범위로 스케일링
            # 입력 데이터도 [-1, 1]로 정규화되어 있어서 일치시킴
            nn.Tanh()
        )

    def forward(self, z):  # 생성자 순전파 함수
        """
        생성자의 순전파 과정
        Args:
            z: 입력 노이즈 벡터 (batch_size, latent_dim)
        Returns:
            생성된 이미지 (batch_size, channels, height, width)
        """
        # 노이즈 벡터를 모델에 통과시켜 평면화된 이미지 생성
        img = self.model(z)  # (batch_size, 784)

        # 생성된 평면화된 이미지를 원래 이미지 형태로 reshape
        # img.size(0)는 배치 크기, *img_shape는 (1, 28, 28)를 언패킹
        img = img.view(img.size(0), *img_shape)  # (batch_size, 1, 28, 28)

        return img  # 생성된 이미지 반환

# ================================
# 7. Discriminator(판별자) 모델 정의
# ================================

class Discriminator(nn.Module):  # 판별자 모델 클래스 정의 (nn.Module 상속)
    def __init__(self):  # 판별자 초기화 함수
        super(Discriminator, self).__init__()  # 부모 클래스(nn.Module) 초기화

        # 여러 레이어를 순차적으로 연결하는 Sequential 모델 구성
        self.model = nn.Sequential(
            # 첫 번째 레이어: 이미지 픽셀 수(784)를 입력받아 512차원으로 변환
            # int(np.prod(img_shape)) = 784 (28*28*1)
            nn.Linear(int(np.prod(img_shape)), 512),

            # LeakyReLU 활성화 함수 (negative slope = 0.2)
            # 일반 ReLU와 달리 음수 영역에서도 작은 기울기 유지
            nn.LeakyReLU(0.2, inplace=True),

            # 두 번째 레이어: 512차원에서 256차원으로 축소
            nn.Linear(512, 256),

            # LeakyReLU 활성화 함수
            nn.LeakyReLU(0.2, inplace=True),

            # 최종 출력 레이어: 256차원에서 1차원으로 축소 (이진 분류)
            nn.Linear(256, 1),

            # Sigmoid 활성화 함수: 출력 값을 [0, 1] 범위의 확률로 변환
            # 0에 가까우면 가짜, 1에 가까우면 진짜로 판단
            nn.Sigmoid(),
        )

    def forward(self, img):  # 판별자 순전파 함수
        """
        판별자의 순전파 과정
        Args:
            img: 입력 이미지 (batch_size, channels, height, width)
        Returns:
            진짜일 확률 (batch_size, 1)
        """
        # 이미지를 1차원 벡터로 펼치기
        # img.size(0)은 배치 크기, -1은 나머지 차원을 자동 계산
        img_flat = img.view(img.size(0), -1)  # (batch_size, 784)

        # 펼쳐진 이미지를 모델에 통과시켜 진짜일 확률 예측
        validity = self.model(img_flat)  # (batch_size, 1)

        return validity  # 이미지의 진짜일 확률 반환 (0~1 사이)

# ================================
# 8. 학습 하이퍼파라미터 설정
# ================================

lr = 0.0002  # 학습률(Learning Rate) 설정 - 가중치 업데이트 보폭

# Adam 옵티마이저의 베타 파라미터 설정
b1 = 0.5   # Adam 옵티마이저의 베타1(beta1) 파라미터 - 1차 모멘트 지수 감쇠율
b2 = 0.999 # Adam 옵티마이저의 베타2(beta2) 파라미터 - 2차 모멘트 지수 감쇠율

latent_dim = 100  # 생성자 입력으로 사용될 잠재 공간(latent space)의 차원 설정

# ================================
# 9. 모델 인스턴스 생성
# ================================

generator = Generator()      # 생성자 모델 인스턴스 생성
discriminator = Discriminator()  # 판별자 모델 인스턴스 생성

# ================================
# 10. 손실 함수 및 옵티마이저 설정
# ================================

# 적대적 손실 함수로 Binary Cross-Entropy Loss 사용
# BCE는 이진 분류 문제에 적합 (진짜 vs 가짜)
adversarial_loss = nn.BCELoss()

# 생성자 학습을 위한 Adam 옵티마이저 설정
optimizer_G = torch.optim.Adam(
    generator.parameters(),  # 생성자의 모든 학습 가능한 파라미터
    lr=lr,                  # 학습률
    betas=(b1, b2)          # 모멘텀 파라미터 튜플
)

# 판별자 학습을 위한 Adam 옵티마이저 설정
optimizer_D = torch.optim.Adam(
    discriminator.parameters(),  # 판별자의 모든 학습 가능한 파라미터
    lr=lr,                      # 학습률
    betas=(b1, b2)              # 모멘텀 파라미터 튜플
)

# ================================
# 11. GPU 사용 설정
# ================================

# CUDA 사용 가능 여부 확인 (GPU 사용 가능하면 True, 아니면 False)
cuda = True if torch.cuda.is_available() else False

if cuda:  # CUDA 사용 가능 시 모델과 손실 함수를 GPU로 이동
    generator.cuda()        # 생성자를 GPU 메모리로 이동
    discriminator.cuda()    # 판별자를 GPU 메모리로 이동
    adversarial_loss.cuda() # 손실 함수를 GPU로 이동

# ================================
# 12. 학습 준비
# ================================

import time  # 시간 측정을 위한 time 모듈 임포트

n_epochs = 20  # 초기 에포크 수 설정 (이후 셀에서 200으로 변경됨)

sample_interval = 2000  # 생성된 이미지를 저장할 반복(iteration) 간격 설정
start_time = time.time()  # 학습 시작 시간 기록

# CUDA 사용 시 GPU 텐서, 아니면 CPU 텐서 사용하도록 설정
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

n_epochs = 200  # 총 학습 에포크 수를 200으로 재설정

# ================================
# 13. 초기 생성 이미지 확인
# ================================

# 학습 시작 전에 초기 생성 이미지 (epoch 0) 출력
print("Showing initial generated images (before training):")

# 25개의 이미지를 생성하기 위한 잠재 공간 노이즈 벡터 생성
# Variable은 자동 미분을 위한 래퍼 (현재는 권장되지 않지만 호환성을 위해 사용)
z = Variable(Tensor(25, latent_dim))

if cuda:
    z = z.cuda()  # CUDA 사용 시 노이즈 벡터를 GPU로 이동

# 생성자 모델을 사용하여 이미지 생성
generated_imgs = generator(z)

# 생성된 이미지 25개를 파일로 저장
# nrow=5: 5x5 그리드로 배열
# normalize=True: 픽셀 값을 [0, 1] 범위로 정규화하여 저장
save_image(generated_imgs.data[:25], "generated_epoch_0_initial.png", nrow=5, normalize=True)

# 저장된 초기 이미지 파일을 주피터 노트북에 출력
display(Image(filename="generated_epoch_0_initial.png"))

print("-" * 50)  # 구분선 출력

# ================================
# 14. 메인 학습 루프
# ================================

# 설정된 에포크 수만큼 반복하며 학습 진행
for epoch in range(n_epochs):

    # 데이터 로더에서 배치 단위로 이미지 가져오기
    # imgs: 실제 이미지 배치, _: 레이블 (사용하지 않음)
    for i, (imgs, _) in enumerate(dataloader):

        # ================================
        # 14.1 타겟 레이블 생성
        # ================================

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 타겟 레이블 생성
        # requires_grad=False: 이 변수들에 대해서는 그래디언트 계산 안 함
        real = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)  # 진짜 이미지의 타겟 레이블 (1.0)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)  # 가짜 이미지의 타겟 레이블 (0.0)

        # 실제 이미지 데이터를 현재 사용 중인 텐서 타입(CPU/GPU)으로 변환
        real_imgs = Variable(imgs.type(Tensor))

        # ================================
        # 14.2 생성자(Generator) 학습
        # ================================

        optimizer_G.zero_grad()  # 생성자 옵티마이저의 기울기 초기화 (이전 배치의 그래디언트 제거)

        # 생성자 입력을 위한 무작위 노이즈 벡터 생성
        # np.random.normal(0, 1, shape): 평균 0, 표준편차 1인 정규분포에서 샘플링
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        if cuda:
            z = z.cuda()  # CUDA 사용 시 노이즈 벡터를 GPU로 이동

        generated_imgs = generator(z)  # 생성자로 가짜 이미지 생성

        # 생성자가 생성한 가짜 이미지가 판별자에게 진짜(1.0)로 인식되도록 손실 계산
        # 생성자의 목표: 판별자를 속여서 가짜 이미지를 진짜로 분류하게 만들기
        g_loss = adversarial_loss(discriminator(generated_imgs), real)

        g_loss.backward()  # 생성자 손실에 대한 역전파 수행 (그래디언트 계산)
        optimizer_G.step()  # 생성자 옵티마이저 스텝 (계산된 그래디언트로 가중치 업데이트)

        # ================================
        # 14.3 판별자(Discriminator) 학습
        # ================================

        optimizer_D.zero_grad()  # 판별자 옵티마이저의 기울기 초기화

        # 판별자가 진짜 이미지를 진짜(1.0)로 올바르게 인식하도록 손실 계산
        real_loss = adversarial_loss(discriminator(real_imgs), real)

        # 판별자가 생성자가 생성한 가짜 이미지를 가짜(0.0)로 올바르게 인식하도록 손실 계산
        # .detach()를 사용하여 생성자 쪽으로는 기울기가 전달되지 않도록 함 (중요!)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)

        # 판별자의 총 손실은 진짜와 가짜 이미지에 대한 손실의 평균
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()  # 판별자 손실에 대한 역전파 수행
        optimizer_D.step()  # 판별자 옵티마이저 스텝 (가중치 업데이트)

        # ================================
        # 14.4 중간 결과 저장
        # ================================

        # 생성된 이미지를 주기적으로 저장
        done = epoch * len(dataloader) + i  # 현재까지 완료된 총 반복(iteration) 수 계산

        if done % sample_interval == 0:  # 설정된 저장 간격(2000 iter)마다 이미지 저장
            # 생성된 이미지 25개를 파일로 저장 (5x5 그리드)
            save_image(generated_imgs.data[:25], f"generated_epoch_{epoch}_iter_{i}.png", nrow=5, normalize=True)

    # ================================
    # 14.5 에포크별 결과 출력
    # ================================

    # 각 에포크 종료 시 손실 값과 소요 시간 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

    # 5 에포크마다 가장 최근에 생성된 이미지 출력
    if (epoch + 1) % 5 == 0:  # 현재 에포크 번호가 5로 나누어 떨어지면 (5, 10, 15, 20...)

        # 해당 에포크에서 저장된 가장 최근 이미지 파일 찾기
        latest_image_path = None

        # 에포크의 마지막 반복에서 저장된 이미지가 '가장 최근'이라고 가정하고 파일 경로 생성
        latest_image_path = f"generated_epoch_{epoch}_iter_{len(dataloader)-1}.png"

        # sample_interval 때문에 마지막 반복에서 이미지가 저장되지 않았을 경우를 대비한 대체 로직
        if not os.path.exists(latest_image_path):
            # 해당 에포크에서 저장된 모든 이미지 파일을 찾아 시간 순으로 정렬
            image_paths = sorted(glob.glob(f"generated_epoch_{epoch}_*.png"))
            if image_paths:
                latest_image_path = image_paths[-1]  # 가장 마지막 파일 선택

        # 가장 최근 이미지 파일이 존재하면 출력
        if latest_image_path and os.path.exists(latest_image_path):
            print(f"\nShowing generated images after Epoch {epoch+1}:")  # 출력 메시지
            display(Image(filename=latest_image_path))  # 이미지 파일을 주피터 노트북에 출력